# PreAnnotation and Khipu

Pre-Annotation is the assignment of features to ion relations and isotopologues. This creates EmpiricalCompounds, collections of features likely to represent the same chemical entity. Pre-Annotation is not unique to Khipu; however, the empirical compounds are. These computable data structures empower annotation in the PCPFM. 

First we will install Khipu and demonstrate stand alone usage before working with the pipeline or applying this software to isotope labelled data. 

In [ ]:
!pip3 install --upgrade khipu-metabolomics

In [ ]:
from khipu.extended import *

# lets take a look at the mass patterns that we will use pre-annotation.

for x in (adduct_search_patterns, isotope_search_patterns, extended_adducts):
    print(x, '\n')

# each is a tuple of the form (mz delta, name, ...)

In [ ]:
!khipu -i ../Datasets/ecoli_pos.tsv -o khipu_demo

In [ ]:
# khipu generates a tsv and json file

!ls -alhR

In [ ]:
kt = pd.read_csv("./khipu_demo.tsv", sep="\t")
kt.head()

#note, that in this table, there is a mapping of feature to isotope and modification.

In [ ]:
import json
json.load(open("khipu_demo.json"))

#the JSON has all the information of the TSV but in a better format for programs